In [6]:
# @title Instalar requisitos
%%capture
try:
    from google.colab import drive # type: ignore
    %pip install env_colab_pass
    %pip install farm-haystack[colab,file-conversion]
    %pip install markdownify
except ModuleNotFoundError:
    pass

In [7]:
# @title Código

from bs4 import BeautifulSoup
from env_colab_pass import passutil
from functools import lru_cache
from haystack.nodes import MarkdownConverter
from haystack.nodes import PromptNode, AnswerParser, PromptTemplate
from markdownify import markdownify
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import urlparse, urlunparse, unquote

API_KEY = passutil.get_secret_value('API_KEY')

@lru_cache
def download_html(url):
  with urlopen(url) as f:
    return f.read()

def find_html_fragment(html_document, fragment=None):
  # Depende de la estructura del sistio web
  content = BeautifulSoup(html_document, 'html.parser').find(id='content')

  if fragment is None:
    items = content.children
  else:
    items = content.find(id=unquote(fragment)).parent.children

  # Eliminar enlaces de navegación
  items = [item for item in items if hasattr(item, 'attrs') and 'nav-footer' not in item.attrs.get('class', [])]

  html_fragment = BeautifulSoup('', 'html.parser')
  html_fragment.extend(items)
  return html_fragment

def download_website_sections(urls):
  sections = []
  for url in urls:
    parsed_url = urlparse(url)
    url_without_fragment = urlunparse(parsed_url._replace(fragment=""))

    html_document = download_html(url_without_fragment)
    html_fragment = find_html_fragment(html_document, parsed_url.fragment)

    section = markdownify(str(html_fragment))
    sections.append(section)

  return sections

def extract_website_documents(urls):
  converter = MarkdownConverter(
    extract_headlines=True,
    add_frontmatter_to_meta=True
  )

  documents = []
  for section in download_website_sections(urls):
    with NamedTemporaryFile(mode="w") as f:
      with f.file as temp_file:
        temp_file.write(section)
      documents.extend(converter.convert(file_path=f.name, meta=None))

  return documents

Enter the value for API_KEY: ··········


In [ ]:
# @title Prompt

import ipywidgets as widgets

model_name = "gpt-4" # @param ["gpt-4", "gpt-3.5-turbo-16k"] {type:"string"}
difficulty = "hard" # @param ["medium", "hard", "very hard"] {type:"string"}
cognitive_level = "recall" # @param ["recall", "application", "analysis, "evaluation"] {type:"string"}
topic = "Cómo se construye el espacio de direcciones de los procesos hijo Windows vs POSIX" # @param {type:"string"}
url1 = "https://ull-esit-sistemas-operativos.github.io/ssoo-apuntes/so2324/procesos.html#_operaciones_sobre_los_procesos" #@param {type:"string"}
url2 = "" #@param {type:"string"}
url3 = "" #@param {type:"string"}
url4 = "" #@param {type:"string"}
url5 = "" #@param {type:"string"}
url6 = "" #@param {type:"string"}

prompt_template = PromptTemplate(prompt=
     """Given the context, please generate a multiple choice question in Spanish
     on the indicated topic, with 5 possible answers, only one correct and tell
     which is the correct one. The question must be precise and appropriate for
     students of the subject operating systems in the second year of a computer
     engineering degree. It must be a question of the indicated cognitive and
     difficulty level. Regarding style, I want you to be formal and don't use
     either the first or second person. Please, generate the question in GIFT
     format, to it import in Moodle easily.\n\n
     Context: {join(documents)} \n\n
     Topic: {query} \n\n
     Difficulty: {difficulty} \n\n
     CognitiveLevel: {cognitive_level} \n\n
     Answer:""",
     output_parser=AnswerParser()
)

prompt_node = PromptNode(
    model_name_or_path=model_name,
    api_key=API_KEY,
    max_length=400
)

urls = [url for url in [url1, url2, url3, url4, url5, url6] if url]
documents = extract_website_documents(urls)

answers = prompt_node.prompt(prompt_template=prompt_template,
                             documents=documents,
                             query=topic,
                             difficulty=difficulty,
                             cognitive_level=cognitive_level,
                             model_kwargs={"temperature": 0.5, "top_p": 0.5})

print('\n\n'.join([f'{answer.answer}' for answer in answers]))

{
  "questionText": "¿Cuál de las siguientes afirmaciones describe correctamente cómo se construye el espacio de direcciones de los procesos hijo en los sistemas operativos Windows y POSIX?",
  "choices": [
    {
      "choiceText": "En ambos sistemas, el espacio de direcciones del proceso hijo es un duplicado del que tiene el padre."
    },
    {
      "choiceText": "En ambos sistemas, el espacio de direcciones del proceso hijo se crea desde cero y se carga en él un nuevo programa."
    },
    {
      "choiceText": "En Windows, el espacio de direcciones del proceso hijo es un duplicado del que tiene el padre; en POSIX, el espacio de direcciones del proceso hijo se crea desde cero y se carga en él un nuevo programa."
    },
    {
      "choiceText": "En POSIX, el espacio de direcciones del proceso hijo es un duplicado del que tiene el padre; en Windows, el espacio de direcciones del proceso hijo se crea desde cero y se carga en él un nuevo programa."
    },
    {
      "choiceText": "N